<a href="https://colab.research.google.com/github/edwinliew10/LIBS-spectroscopic-analysis/blob/main/Electron%20Density%20Calculation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import pandas as pd
import plotly.express as px
import math
import plotly.graph_objects as go
import numpy as np
from scipy.optimize import curve_fit
import sigfig

In [6]:
from google.colab import files
import io
uploaded = files.upload()

Saving 0.996.csv to 0.996.csv


In [7]:
df = pd.read_csv(io.BytesIO(uploaded['0.996.csv']))
graphx = df.wavelength
graphy = df.intensity
fig = px.line(df, x=np.asarray(df.wavelength), y= np.asarray(df.intensity),
              labels=dict(x="Wavelength (nm)", y="Intensity (counts)"))
fig.update_layout(xaxis_range=[195,900])

fig.show()

In [8]:
#identify selection range
range1 =list(graphx).index(805.85)  #1st point
range2 =list(graphx).index(812.32)  #last point
def isolation():
    select=slice(range1,range2)
    selectx=graphx[select]
    selecty=graphy[select]
    fig = px.scatter(df, 
                  x=selectx, 
                  y=selecty,
                  labels=dict(x="Wavelength", y="Intensity")) 
    fig.show()
    return 
isolation()

In [9]:
selectx=graphx[2362:2389]
selecty=graphy[2362:2389]
variation=(812.32-805.85)/1000 #get thousands of data to make graph smooth
Selectx = np.array(selectx)
length=len(selectx)
meany = np.sum(selecty)/length

fig = px.scatter(df, 
              x=selectx, 
              y=selecty,
              labels=dict(x="Wavelength", y="Intensity"))

#Voigt
def voigt(selectx,b,c,w,a,mu):
    return b+a*(mu*(2/math.pi)*(w/(4*(selectx-c)**2+w**2))+(1-mu)*(math.sqrt(4*math.log(2))/(math.sqrt(math.pi)*w))*np.exp(-(4*math.log(2)/w**2)*(selectx-c)**2))  
x=selectx.values
y=selecty.values
g=[5,808,0.5,809,0.5]
c,cov=curve_fit(voigt,x,y,g)
C=np.asarray(c)
Voi_b=C[0]
Voi_c=C[1]
Voi_w=C[2]
Voi_a=C[3]
Voi_mu=C[4]
new_intensity2=[]
new_wavelength2=[]
for fitting in selecty:
    new1=Voi_b+Voi_a*(Voi_mu*(2/math.pi)*(Voi_w/(4*(selectx-Voi_c)**2+Voi_w**2))+(1-Voi_mu)*(math.sqrt(4*math.log(2))/(math.sqrt(math.pi)*Voi_w))*np.exp(-(4*math.log(2)/Voi_w**2)*(selectx-Voi_c)**2))  
    new_intensity2.append(new1)
    break
    
New_intensity2 = np.asarray(new_intensity2)
for x in selecty:
    ymean2=(selecty-meany)**2
    minise2=(New_intensity2-meany)**2
    break
totalymean2=np.sum(ymean2)
totalminise2=np.sum(minise2)
r_squ2=(totalminise2/totalymean2)
    
#Lorentz
def lorentz(selectx,b,c,w,a):
    return b+(2*a/math.pi)*(w/(4*(selectx-c)**2+w**2))
x=selectx.values
y=selecty.values
g=[5,808,0.5,809]
c,cov=curve_fit(lorentz,x,y,g)
C=np.asarray(c)

lor_b=C[0]
lor_c=C[1]
lor_w=C[2]
lor_a=C[3]
new_intensity3=[]
new_wavelength3=[]
for fitting in selecty:
    new1=lor_b+(2*lor_a/math.pi)*(lor_w/(4*(Selectx-lor_c)**2+lor_w**2))
    new_intensity3.append(new1)
    break
New_intensity3=np.asarray(new_intensity3)
for x in selecty:
    ymean3=(selecty-meany)**2
    minise3=(New_intensity3-meany)**2
    break
totalymean3=np.sum(ymean3)
totalminise3=np.sum(minise3)
r_squ3=(totalminise3/totalymean3)

#GaussianAmp
def gaussian(selectx,b,c,w,a):
    return b+a*np.exp(-0.5*((selectx-c)/w)**2)
x=selectx.values
y=selecty.values
g=[33,808,0.2,840]
c,cov=curve_fit(gaussian,x,y,g)
C=np.asarray(c)
fittingWavelength=805.85
gau_b=C[0]
gau_c=C[1]
gau_w=C[2]
gau_a=C[3]
new_intensity4=[]
new_wavelength4=[]
for fitting in selecty:
    new1=gau_b+gau_a*np.exp(-0.5*(((selectx-gau_c)/gau_w)**2))
    new_intensity4.append(new1)
    break

New_intensity4=np.asarray(new_intensity4)
for x in selecty:
    ymean4=(selecty-meany)**2
    minise4=(New_intensity4-meany)**2
    break
totalymean4=np.sum(ymean4)
totalminise4=np.sum(minise4)
r_squ4=(totalminise4/totalymean4)

if r_squ2>r_squ3:
    best=r_squ2
else:
    best=r_squ3
    if best<r_squ4:
        best=r_squ4
if best==r_squ2:
    #Voigt
    New_intensity2 = np.asarray(new_intensity2)
    for x in selecty:
        ymean2=(selecty-meany)**2
        minise2=(New_intensity2-meany)**2
        break
    totalymean2=np.sum(ymean2)
    totalminise2=np.sum(minise2)
    r_squ2=(totalminise2/totalymean2)

    fittingWavelength=805.85
    new_intensity2=[]
    new_wavelength2=[]
    while fittingWavelength < 812.32:
        new1=Voi_b+Voi_a*(Voi_mu*(2/math.pi)*(Voi_w/(4*(fittingWavelength-Voi_c)**2+Voi_w**2))+(1-Voi_mu)*(math.sqrt(4*math.log(2))/(math.sqrt(math.pi)*Voi_w))*np.exp(-(4*math.log(2)/Voi_w**2)*(fittingWavelength-Voi_c)**2))  
        new_intensity2.append(new1)
        new_wavelength2.append(fittingWavelength)
        fittingWavelength += variation

    fig = px.scatter(df, 
                x=selectx, 
                y=selecty,
                labels=dict(x="Wavelength", y="Intensity"))
    fig.add_trace(go.Scatter(x=new_wavelength2, y=new_intensity2,
                        mode='lines',name='Voigt'))
    #half peak
    peak=new_intensity2[0]
    for maximum in new_intensity2:
        if maximum>peak:
            peak=maximum
    value=peak/2
    array=[value]*499
    array=np.asarray(array)

    #LEFT GRAPH
    leftHalf=new_intensity2[0:499]
    def nearest(leftHalf,value):
        leftFull=(np.abs(leftHalf-array)).argmin()
        return leftHalf[leftFull]
    leftHalf=np.asarray(leftHalf)
    appro=nearest(leftHalf,array)
    x1=list(new_intensity2).index(appro)
    full1=new_wavelength2[x1]

    #RIGHT GRAPH
    array=[value]*500
    array=np.asarray(array)
    rightHalf=new_intensity2[500:1000]
    def nearest(rightHalf,value):
        rightFull=(np.abs(rightHalf-array)).argmin()
        return rightHalf[rightFull]
    appro=nearest(rightHalf,array)
    x2=list(new_intensity2).index(appro)
    full2=new_wavelength2[x2]
    FWHM=full2-full1
    density=(FWHM/(2*0.00142))*10E16
    fig.show()
    print("R-Squared :",round(r_squ2,5),"\n","\bElectron density= ",sigfig.round(density,6),"cm³")
else:  
    if best==r_squ3:
        #Lorentz
        fittingWavelength=805.85
        new_intensity3=[]
        new_wavelength3=[]
        while fittingWavelength < 812.32:
            new1=lor_b+(2*lor_a/math.pi)*(lor_w/(4*(fittingWavelength-lor_c)**2+lor_w**2)) 
            new_intensity3.append(new1)
            new_wavelength3.append(fittingWavelength)
            fittingWavelength += variation

        fig = px.scatter(df, 
                      x=selectx, 
                      y=selecty,
                      labels=dict(x="Wavelength", y="Intensity"))
        fig.add_trace(go.Scatter(x=new_wavelength3, y=new_intensity3,
                            mode='lines',name='Lorentz'))
        #half peak
        peak=new_intensity3[0]
        for maximum in new_intensity3:
            if maximum>peak:
                peak=maximum
        value=peak/2
        array=[value]*499
        array=np.asarray(array)

        #LEFT GRAPH
        leftHalf=new_intensity3[0:499]
        def nearest(leftHalf,value):
            leftFull=(np.abs(leftHalf-array)).argmin()
            return leftHalf[leftFull]
        leftHalf=np.asarray(leftHalf)
        appro=nearest(leftHalf,array)
        x1=list(new_intensity3).index(appro)
        full1=new_wavelength3[x1]

        #RIGHT GRAPH
        array=[value]*500
        array=np.asarray(array)
        rightHalf=new_intensity3[500:1000]
        def nearest(rightHalf,value):
            rightFull=(np.abs(rightHalf-array)).argmin()
            return rightHalf[rightFull]
        appro=nearest(rightHalf,array)
        x2=list(new_intensity3).index(appro)
        full2=new_wavelength3[x2]
        FWHM=full2-full1
        density=(FWHM/(2*0.00142))*10E16
        fig.show()
        print("R-Squared :",round(r_squ3,5),"\n","\bElectron density =",sigfig.round(density,6),"cm³")
        
    else:
        #GaussianAmp
        fittingWavelength=805.85
        new_intensity4=[]
        new_wavelength4=[]
        while fittingWavelength < 812.32:
            new1=gau_b+gau_a*math.exp(-0.5*((fittingWavelength-gau_c)/gau_w)**2)
            new_intensity4.append(new1)
            new_wavelength4.append(fittingWavelength)
            fittingWavelength += variation

        fig = px.scatter(df, 
                      x=selectx, 
                      y=selecty,
                      labels=dict(x="Wavelength", y="Intensity"))
        fig.add_trace(go.Scatter(x=new_wavelength4, y=new_intensity4,
                            mode='lines',name='Gaussian'))
        #half peak
        peak=new_intensity4[0]
        for maximum in new_intensity4:
            if maximum>peak:
                peak=maximum
        value=peak/2
        array=[value]*499
        array=np.asarray(array)

        #LEFT GRAPH
        leftHalf=new_intensity4[0:499]
        def nearest(leftHalf,value):
            leftFull=(np.abs(leftHalf-array)).argmin()
            return leftHalf[leftFull]
        leftHalf=np.asarray(leftHalf)
        appro=nearest(leftHalf,array)
        x1=list(new_intensity4).index(appro)
        full1=new_wavelength4[x1]

        #RIGHT GRAPH
        array=[value]*500
        array=np.asarray(array)
        rightHalf=new_intensity4[500:1000]
        def nearest(rightHalf,value):
            rightFull=(np.abs(rightHalf-array)).argmin()
            return rightHalf[rightFull]
        appro=nearest(rightHalf,array)
        x2=list(new_intensity4).index(appro)
        full2=new_wavelength4[x2]
        FWHM=full2-full1
        density=(FWHM/(2*0.00142))*10E16
        fig.show()
        print("R-Squared :",round(r_squ4,5),"\n","\bElectron density= ",sigfig.round(density,6),"cm³")

R-Squared : 0.99612 
 Electron density=  3.07553e+19 cm³
